In [1]:
!kill $(lsof -t -i:7860)


kill: usage: kill [-s sigspec | -n signum | -sigspec] pid | jobspec ... or kill -l [sigspec]


In [2]:
# ======================================================
# 🤖 AI Customer Support Bot (Groq + LangChain + Chroma + Gradio + ngrok)
# ======================================================

# Step 1: Install dependencies
!pip install langchain langchain-core langchain-community langchain-groq gradio pypdf chromadb sentence_transformers nest_asyncio pyngrok > /dev/null

# Step 2: Imports
from langchain_groq import ChatGroq
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain.embeddings import SentenceTransformerEmbeddings
import gradio as gr
import tempfile
import os
import nest_asyncio
from pyngrok import ngrok

# Step 3: Apply nest_asyncio (needed for Colab)
nest_asyncio.apply()

# Step 4: Set your ngrok auth token
ngrok.set_auth_token("2yLZwyu0M6abrDAXgjcaaDqArlL_5PtJBVqHHxyu9upkUV3UH")

# Step 5: Initialize LLM
llm = ChatGroq(
    temperature=0,
    groq_api_key="gsk_...",  # 🔑 Replace with your Groq API key
    model_name="llama-3.3-70b-versatile"
)

# Step 6: Global objects
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
vectorstore = None  # will hold Chroma vectorstore
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# ======================================================
# Optional: Upload Knowledge Base (PDF)
# ======================================================
def upload_pdf(file):
    global vectorstore
    if file is None:
        return "⚠️ Please upload a PDF file."
    with tempfile.NamedTemporaryFile(delete=False, suffix=".pdf") as tmp:
        tmp.write(file.read())
        tmp_path = tmp.name

    loader = PyPDFLoader(tmp_path)
    docs = loader.load()
    vectorstore = Chroma.from_documents(docs, embedding_function)
    os.remove(tmp_path)
    return f"✅ Knowledge base '{file.name}' uploaded successfully!"

# ======================================================
# Chatbot Function with Context Memory & Escalation
# ======================================================
def chat_with_bot(message, history):
    global vectorstore, memory

    if vectorstore:
        retriever = vectorstore.as_retriever(search_kwargs={"k": 3})
    else:
        retriever = None

    qa_chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=retriever,
        memory=memory,
        return_source_documents=False
    )

    response = qa_chain.invoke({"question": message})
    answer = response["answer"]

    # Escalation simulation
    if any(phrase in answer.lower() for phrase in ["i don't know", "not sure", "no information"]):
        answer += "\n\n🔺 Escalating this query to a human support agent."

    return answer

# ======================================================
# Launch Gradio UI with ngrok
# ======================================================
with gr.Blocks() as demo:
    gr.Markdown("## 🤖 AI Customer Support Bot")
    gr.Markdown(
        "💡 Features:\n"
        "- Contextual memory of conversation\n"
        "- Escalation simulation for unanswered queries\n"
        "- Optional PDF knowledge base upload"
    )

    kb_upload = gr.File(label="📚 Upload Knowledge Base (PDF)")
    upload_btn = gr.Button("Upload")
    upload_output = gr.Textbox(label="Upload Status")

    upload_btn.click(upload_pdf, inputs=kb_upload, outputs=upload_output)

    gr.ChatInterface(fn=chat_with_bot, title="AI Support Chat").render()

# Step 1: Expose Gradio via ngrok
public_url = ngrok.connect(7860)
print("🚀 Gradio frontend running at:", public_url)

# Step 2: Launch Gradio app
demo.launch(server_name="0.0.0.0", server_port=7860)


/tmp/ipython-input-4280712122.py:35: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datas

🚀 Gradio frontend running at: NgrokTunnel: "https://6c31e3405f2d.ngrok-free.app" -> "http://localhost:7860"
It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://de38b98f6451d4d347.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
